In [35]:
%matplotlib qt5
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
def smooth(y, box_pts):
    box = np.ones(box_pts) / box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [2]:
base_path = Path('/home/harsh/AutoGuider Data/')

In [14]:
ra_filename = base_path / 'RA_Calib_10032021_0950.txt'

In [15]:
dec_filename = base_path / 'DEC_Calib_10032021_0948.txt'

In [16]:
encoding='utf-16-le'

In [17]:
ra_f = open(ra_filename, encoding=encoding)

In [18]:
ra_lines = ra_f.readlines()

In [19]:
ra_lines

['\ufeffMode Select (Enter e for exit): \n',
 'Enter 1 for RA test.\n',
 'Enter 2 for DEC test.\n',
 'Enter 3 for Motor test.\n',
 'Enter 4 for Serial loopback test.\n',
 'Enter 5 for RA close loop calibration.\n',
 'Enter 6 for DEC close loop calibration.\n',
 'Enter 7 for Monitoring RA voltage drift.\n',
 'Enter 8 for Monitoring DEC voltage drift.\n',
 'Enter 9 for Monitoring RA and DEC voltage drift.\n',
 'Enter 10 for Close loop operation.\n',
 'Mode Selected:5\n',
 'Enter 0 for Individual Mode, 1 for coupled mode\n',
 'Samples Input Mode: 1\n',
 'Enter the COM Port number: \n',
 'COM Port: \\\\.\\COM1\n',
 'Successfully ran itoa\n',
 'byteSize: 8\n',
 'stopBits: 0\n',
 'parity: 0\n',
 'statusGetCommState: 1\n',
 'Enter the raFrequency of motor: \n',
 'Enter the total counts to move: \n',
 'Enter the step size in counts: \n',
 'Enter the motor direction: \n',
 'Command to Motor:<2E>\n',
 '\n',
 'Command to Motor:<2F350>\n',
 '\n',
 'Creating DAQ handle : True\n',
 'Device Name: Dev

In [20]:
ra_data_lines = [line for line in ra_lines if line[0].isdigit()]

ra_data = np.loadtxt(ra_data_lines)

In [21]:
ra_data.shape

(30, 401)

In [23]:
ra_data[:, 1:].shape

(30, 400)

In [24]:
ra_data = ra_data[:, 1:]

In [25]:
ra_data_1 = ra_data.reshape(
        ra_data.shape[0], ra_data.shape[1] // 2, 2)[:, :, 0]

In [26]:
ra_data_2 = ra_data.reshape(
        ra_data.shape[0], ra_data.shape[1] // 2, 2)[:, :, 1]

In [27]:
ra_mean_data_1 = np.mean(ra_data_1, axis=1)

In [28]:
ra_mean_data_2 = np.mean(ra_data_2, axis=1)

In [30]:
ra_data_1.shape

(30, 200)

In [80]:
plt.plot(ra_mean_data_1)

In [120]:
ra_indices_1 = [2, 10]

In [121]:
ra_direction = -1

In [122]:
ra_a1, ra_b1 = np.polyfit(
        ra_direction * 100 * np.arange(
            ra_indices_1[1] - ra_indices_1[0]),
        smooth(ra_mean_data_1, 5)[np.arange(ra_indices_1[0], ra_indices_1[1])],
        1
    )

In [123]:
ra_y1 = ra_a1 * ra_direction * 100 * np.arange(
        ra_indices_1[1] - ra_indices_1[0]
    ) + ra_b1

In [124]:
ra_indices_2 = [5, 15]

In [125]:
ra_a2, ra_b2 = np.polyfit(
        ra_direction * 100 * np.arange(
            ra_indices_2[1] - ra_indices_2[0]),
        smooth(ra_mean_data_2, 5)[np.arange(ra_indices_2[0], ra_indices_2[1])],
        1
    )

In [126]:
ra_y2 = ra_a2 * ra_direction * 100 * np.arange(
        ra_indices_2[1] - ra_indices_2[0]
    ) + ra_b2

In [114]:
dec_f = open(dec_filename, encoding=encoding)

In [115]:
dec_lines = dec_f.readlines()

In [116]:
dec_data_lines = [line for line in dec_lines if line[0].isdigit()]

In [117]:
dec_data = np.loadtxt(dec_data_lines)

In [47]:
dec_data.shape

(30, 401)

In [48]:
dec_data = dec_data[:, 1:]

In [49]:
dec_data_1 = dec_data.reshape(
        dec_data.shape[0], dec_data.shape[1] // 2, 2)[:, :, 0]

In [50]:
dec_data_2 = dec_data.reshape(
        dec_data.shape[0], dec_data.shape[1] // 2, 2)[:, :, 1]

In [51]:
dec_mean_data_1 = np.mean(dec_data_1, axis=1)

In [52]:
dec_mean_data_2 = np.mean(dec_data_2, axis=1)

In [57]:
plt.plot(dec_mean_data_2)

In [70]:
dec_direction = 1

In [71]:
dec_indices_1 = [0, 6]

In [72]:
dec_a1, dec_b1 = np.polyfit(
        dec_direction * 100 * np.arange(
            dec_indices_1[1] - dec_indices_1[0]),
        smooth(
            dec_mean_data_1, 5)[np.arange(dec_indices_1[0], dec_indices_1[1])],
        1
    )

In [73]:
dec_y1 = dec_a1 * dec_direction * 100 * np.arange(
        dec_indices_1[1] - dec_indices_1[0]
    ) + dec_b1

In [74]:
dec_indices_2 = [15, 25]

In [75]:
dec_a2, dec_b2 = np.polyfit(
        dec_direction * 100 * np.arange(
            dec_indices_2[1] - dec_indices_2[0]),
        smooth(
            dec_mean_data_2, 5)[np.arange(dec_indices_2[0], dec_indices_2[1])],
        1
    )

In [76]:
dec_y2 = dec_a2 * dec_direction * 100 * np.arange(
        dec_indices_2[1] - dec_indices_2[0]
    ) + dec_b2

In [127]:
calib = np.array([[ra_a1, ra_a2], [dec_a2, dec_a1]])

In [129]:
calib

array([[ 0.44349956, -0.03181819],
       [-0.13775145,  0.53727753]])

In [130]:
np.linalg.inv(calib)

array([[2.29704586, 0.13603368],
       [0.58893473, 1.89611288]])